# Ein Hybridmodell mit LSTM-AE, DBSCAN und IF

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import classification_report
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import (
    adjusted_rand_score,
    homogeneity_score,
    fowlkes_mallows_score
)
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import logging
from tensorflow.keras.layers import TimeDistributed, Dense, LSTM, Dropout, LayerNormalization, Input, Add, Bidirectional, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score
import pandas as pd
import os
import sys
import logging
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
from keras.layers import LSTM, Dropout, LayerNormalization, Add, TimeDistributed, Dense
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report

2025-07-16 15:05:26.144714: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752671126.199418  143721 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752671126.214648  143721 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752671126.245266  143721 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752671126.245296  143721 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752671126.245299  143721 computation_placer.cc:177] computation placer alr

## 1. Vorbereitung

#### Cuda auslassen, weil DGX verwendet wird

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#### Daten laden

In [3]:
data_train = pd.read_json("train_logs.json", lines=False)
data_test = pd.read_json("test_logs.json", lines=False)
data_val = pd.read_json("val_logs.json", lines=False)
print(data_test.head())

                         timestamp log_level             category  \
0 2025-07-14 04:48:46.534011+02:00      warn  org.keycloak.events   
1 2025-07-14 06:37:18.492909+02:00      warn  org.keycloak.events   
2 2025-07-15 14:14:20.683869+02:00       NaN                  NaN   
3 2025-07-15 18:22:57.415589+02:00       NaN                  NaN   
4 2025-07-15 18:30:15.424422+02:00      warn  org.keycloak.events   

          type  realmId realmName                clientId        ipAddress  \
0  LOGIN_ERROR    bosch     bosch               admin-cli  198.123.114.208   
1  LOGIN_ERROR  telekom   telekom               admin-cli  203.142.240.178   
2          NaN      NaN       NaN                     NaN              NaN   
3          NaN      NaN       NaN                     NaN              NaN   
4  LOGIN_ERROR    bosch     bosch  security-admin-console    203.29.25.248   

                 error username  \
0  invalid_credentials      Tom   
1  invalid_credentials     Yara   
2          

#### NaN-Werte bereinigen

In [4]:
X_train_full = pd.DataFrame(data_train)
X_test_full = pd.DataFrame(data_test)
X_val_full = pd.DataFrame(data_val)

X_train_full = X_train_full.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
X_test_full = X_test_full.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
X_val_full = X_val_full.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

y_test_full = data_test["label"]

#### Logging

In [ ]:
logfile = open("output_log.txt", "w")
logging.basicConfig(stream=logfile, level=logging.INFO)

sys.stdout = logfile
sys.stderr = logfile

print("Das ist eine Print-Ausgabe.")
logging.info("Das ist eine Log-Nachricht.")

2025-07-16 15:05:31.339044: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-07-16 15:05:31.339107: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:167] env: CUDA_VISIBLE_DEVICES="-1"
2025-07-16 15:05:31.339118: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-07-16 15:05:31.339125: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-07-16 15:05:31.339132: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: Ueay-Ubuntu
2025-07-16 15:05:31.339137: I external/local_xla/xla/stream_executor/cuda/c

#### Automatisch alles in Numerische oder Kategorische Daten einteilen

In [6]:
def auto_encode_features(logs, one_hot_numeric=False, 
                         label_encoders=None, onehot_encoders=None, fit=True):
    if logs is None or len(logs) == 0:
        return [], {}, {}

    if hasattr(logs, "to_dict"):
        logs = logs.to_dict(orient='records')

    if label_encoders is None:
        label_encoders = {}
    if onehot_encoders is None:
        onehot_encoders = {}

    sample = logs[0]
    all_keys = sample.keys()
    encoded_logs = []

    for log in logs:
        encoded = {}
        for key in all_keys:
            val = log.get(key)

            if val is None:
                continue

            if isinstance(val, (int, float)) and not isinstance(val, bool):
                if one_hot_numeric:
                    if key not in onehot_encoders and fit:
                        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
                        values = np.array([[l.get(key)] for l in logs if l.get(key) is not None])
                        encoder.fit(values)
                        onehot_encoders[key] = encoder

                    if key in onehot_encoders:
                        enc = onehot_encoders[key].transform([[val]])[0]
                        for i, v in enumerate(enc):
                            encoded[f"{key}_{i}"] = v
                else:
                    encoded[key] = val

            elif isinstance(val, str):
                if key not in label_encoders and fit:
                    le = LabelEncoder()
                    values = [l.get(key) for l in logs if l.get(key) is not None]
                    le.fit(values)
                    label_encoders[key] = le

                if key in label_encoders:
                    encoded[key] = label_encoders[key].transform([val])[0]

            elif isinstance(val, list):
                for i, item in enumerate(val):
                    label = f"{key}_{i}"
                    if label not in label_encoders and fit:
                        le = LabelEncoder()
                        values = [item for l in logs for item in l.get(key, [])]
                        le.fit(values)
                        label_encoders[label] = le

                    if label in label_encoders:
                        encoded[label] = label_encoders[label].transform([item])[0]
        encoded_logs.append(encoded)

    return encoded_logs, label_encoders, onehot_encoders


#### Werte aus vorheriger Funktion vereinigen

In [7]:
def dicts_to_feature_matrix(encoded_logs):
    feature_names = sorted({key for d in encoded_logs for key in d.keys()})

    X = np.zeros((len(encoded_logs), len(feature_names)), dtype=np.float32)

    for i, d in enumerate(encoded_logs):
        for j, feat in enumerate(feature_names):
            if feat in d:
                X[i, j] = d[feat]

    return X, feature_names

In [8]:
logs_train = X_train_full.to_dict(orient='records')
logs_test = X_test_full.to_dict(orient='records')
logs_val = X_val_full.to_dict(orient='records')

encoded_logs, label_encoders, onehot_encoders = auto_encode_features(logs_train, 
                                                one_hot_numeric=False, fit=True)
X_train, feature_names = dicts_to_feature_matrix(encoded_logs)

encoded_test_logs, _, _ = auto_encode_features(logs_test, one_hot_numeric=False,
                                               label_encoders=label_encoders, onehot_encoders=onehot_encoders, fit=False)
X_test, _ = dicts_to_feature_matrix(encoded_test_logs)

encoded_val_logs, _, _ = auto_encode_features(logs_val, one_hot_numeric=False,
                                              label_encoders=label_encoders, onehot_encoders=onehot_encoders, fit=False)
X_val, _ = dicts_to_feature_matrix(encoded_val_logs)

#### Skalierung

In [9]:
# Werte skalieren
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_full.to_numpy()).astype(np.float32)
X_test_scaled = scaler.transform(X_test_full.reindex(columns=X_train_full.columns, fill_value=0).to_numpy()).astype(np.float32)
X_val_scaled = scaler.transform(X_val_full.reindex(columns=X_train_full.columns, fill_value=0).to_numpy()).astype(np.float32)

#### PCA zur Dimensionsreduktion

In [10]:
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
X_val_pca = pca.transform(X_val_scaled)

### Parameter

In [11]:
seq_length = 32
batch_size = 128

encoder_layers = [1024, 512, 256]
decoder_layers = [256, 512, 1024]

dropout_rate = 0.2

timesteps = seq_length
n_features = X_train_pca.shape[1]
learning_rate = 0.0001


### Sequenzen

In [12]:
# Generatoren bleiben gleich
class SequenceToSequenceGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, seq_length, batch_size):
        self.data = data
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.indices = np.arange(len(data) - seq_length)

    def __len__(self):
        return (len(self.indices) + self.batch_size - 1) // self.batch_size

    def __getitem__(self, idx):
        batch_idx = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        if len(batch_idx) == 0:
            batch_idx = self.indices[-self.batch_size:]
        X_batch = np.array([self.data[i:i + self.seq_length] for i in batch_idx])
        return X_batch, X_batch

train_gen = SequenceToSequenceGenerator(X_train_pca, seq_length, batch_size)
val_gen = SequenceToSequenceGenerator(X_val_pca, seq_length, batch_size)
test_gen = SequenceToSequenceGenerator(X_test_pca, seq_length, batch_size)

## 2. LSTM-AE implementieren

In [13]:
from tensorflow.keras import backend as K

inputs = Input(shape=(timesteps, n_features))
x = inputs

def residual_lstm_block(x, units, dropout_rate=0.2):
    shortcut = x
    x = LSTM(units, return_sequences=True)(x)
    x = Dropout(dropout_rate)(x)
    x = LayerNormalization()(x)

    print("Shape x:", K.int_shape(x))
    print("Shape shortcut:", K.int_shape(shortcut))
    
    if K.int_shape(shortcut)[-1] != units:
        shortcut = TimeDistributed(Dense(units))(shortcut)
        print("Shortcut after Dense:", K.int_shape(shortcut))
    
    x = Add()([x, shortcut])
    return x

inputs = Input(shape=(timesteps, n_features))
x = inputs

for units in encoder_layers:
    x = residual_lstm_block(x, units, dropout_rate=dropout_rate)

encoded = Bidirectional(LSTM(encoder_layers[-1]))(x)
print("Encoded shape:", K.int_shape(encoded))  # (None, units*2)

decoded = RepeatVector(timesteps)(encoded)
print("Decoded shape:", K.int_shape(decoded))  # (None, timesteps, units*2)

x = decoded
for units in decoder_layers:
    x = LSTM(units, return_sequences=True)(x)
    x = Dropout(dropout_rate)(x)
    x = LayerNormalization()(x)
    print("Decoder layer output shape:", K.int_shape(x))

outputs = TimeDistributed(Dense(n_features, activation='linear'))(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate), loss='mse')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 32, 10)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32, 1024)  │  4,239,360 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32, 1024)  │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 32, 1024)  │      2,048 │ dropout[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 32, 1024)  │     11,264 │ input_layer_1[0]… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 32, 1024)  │          0 │ layer_normalizat… │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32, 512)   │  3,147,776 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32, 512)   │          0 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 32, 512)   │      1,024 │ dropout_1[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 32, 512)   │    524,800 │ add[0][0]         │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 32, 512)   │          0 │ layer_normalizat… │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 32, 256)   │    787,456 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 32, 256)   │          0 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 32, 256)   │        512 │ dropout_2[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 32, 256)   │    131,328 │ add_1[0][0]       │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 32, 256)   │          0 │ layer_normalizat… │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 512)       │  1,050,624 │ add_2[0][0]       │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 32, 512)   │          0 │ bidirectional[0]… │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 32, 256)   │    787,456 │ repeat_vector[0]

 Total params: 18,567,946 (70.83 MB)

 Trainable params: 18,567,946 (70.83 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stop])

### Umwandlung der Daten für nächste Modelle: Rekonstruktionsfehler berechnen

In [ ]:
def get_reconstruction_errors(gen, model):
    errors = []
    for i in range(len(gen)):
        X_batch, _ = gen[i]
        pred = model.predict(X_batch, verbose=0)
        batch_errors = np.mean(np.square(X_batch - pred), axis=(1, 2))
        errors.extend(batch_errors)
    return np.array(errors)

train_errors = get_reconstruction_errors(train_gen, model)
reconstruction_errors = get_reconstruction_errors(test_gen, model)
threshold = np.percentile(train_errors, 95)

y_test_seq = np.array([y_test_full[i + seq_length - 1] for i in range(len(reconstruction_errors))])
true_labels = y_test_seq.astype(int)
fpr, tpr, thresholds = roc_curve(true_labels, reconstruction_errors)
optimal_idx = (tpr - fpr).argmax()
optimal_threshold = thresholds[optimal_idx]

y_pred = (reconstruction_errors > optimal_threshold).astype(int)

precision, recall, f1, _ = precision_recall_fscore_support(true_labels, y_pred, average='binary')
roc_auc = roc_auc_score(true_labels, reconstruction_errors)
logging.info(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1-Score: {f1:.3f}, ROC-AUC: {roc_auc:.3f}")

In [ ]:
# Encoder-Model bauen
encoder = tf.keras.Model(inputs, encoded)

# Batchweise encodieren, um RAM zu sparen
X_encoded_batches = []
for i in range(len(test_gen)):
    X_batch, _ = test_gen[i]
    encoded_batch = encoder.predict(X_batch, batch_size=len(X_batch), verbose=0)
    X_encoded_batches.append(encoded_batch)
X_encoded_new = np.vstack(X_encoded_batches)

scaler_enc = StandardScaler()
X_encoded_scaled = scaler_enc.fit_transform(X_encoded_new)
X_encoded_scaled = X_encoded_scaled[-len(reconstruction_errors):]

scaler_err = StandardScaler()
reconstruction_errors_reshaped = reconstruction_errors.reshape(-1, 1)
reconstruction_errors_scaled = scaler_err.fit_transform(reconstruction_errors_reshaped)
hybrid_features = np.hstack((X_encoded_scaled, reconstruction_errors_scaled))

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(
    hybrid_features,
    true_labels,
    test_size=0.2,
    random_state=42,
    stratify=true_labels
)

# Dann: 12.5 % von 80 % ergibt 10 % des gesamten Datensatzes für Validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp,
    y_temp,
    test_size=0.125,  # 0.125 * 0.8 ≈ 0.10 vom Gesamtdatensatz
    random_state=42,
    stratify=y_temp
)

# Modell 1

## Isolation Forest

In [ ]:
# Trainiere nur auf Trainingsdaten
iforest = IsolationForest(random_state=42)
#iforest.fit(X_train)

# Vorhersage auf VALIDIERUNGSDATEN
val_preds = iforest.fit_predict(X_val)
val_anomaly = (val_preds == -1).astype(int)

report_iforest = classification_report(y_val, val_anomaly)
logging.info("Isolation Forest - Validation Report:\n" + report_iforest)

### IF MCC

In [ ]:
mcc = matthews_corrcoef(y_val, val_anomaly)
logging.info("Matthews Correlation Coefficient: %f", mcc)

### IF Balanced Accuracy

In [ ]:
balanced_acc = balanced_accuracy_score(y_val, val_anomaly)
logging.info("Balanced Accuracy: %f", balanced_acc)

### IF AUC-Kurven

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt

# Entscheidungsscores holen (höher = normal → invertieren für Anomalie)
val_scores = -iforest.decision_function(X_val)

# ROC-Kurve und AUC
fpr, tpr, _ = roc_curve(y_val, val_scores)
roc_auc = roc_auc_score(y_val, val_scores)

# PR-Kurve und Average Precision
precision, recall, _ = precision_recall_curve(y_val, val_scores)
pr_auc = average_precision_score(y_val, val_scores)

# Plotten
plt.figure(figsize=(12, 5))

# ROC
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Validation)")
plt.legend()

# PR
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f"AP = {pr_auc:.3f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Validation)")
plt.legend()

plt.tight_layout()
plt.show()


### IF: Anomaly Scores berechnen

In [ ]:
scores = iforest.decision_function(hybrid_features)
print("Scores:", scores[:10])

# Modell 2

## One-Class SVM

In [ ]:
# Training
ocsvm = OneClassSVM(kernel="rbf", gamma='auto', nu=0.05)  # nu ~ erwarteter Anteil Anomalien
#ocsvm.fit(X_train)

val_preds = ocsvm.fit_predict(X_val)
val_anomaly = (val_preds == -1).astype(int)

report_ocsvm = classification_report(y_val, val_anomaly)
logging.info("One-Class SVM - Validation Report:\n" + report_ocsvm)

### OCSVM MCC

In [ ]:
mcc = matthews_corrcoef(y_val, val_anomaly)
logging.info("Matthews Correlation Coefficient: %f", mcc)

### OCSVM Balanced Accuracy

In [ ]:
balanced_acc = balanced_accuracy_score(y_val, val_anomaly)
logging.info("Balanced Accuracy: %f", balanced_acc)

### OCSVM AUC-Kurven

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt

# Anomalie-Scores holen (höher = normal, deshalb invertieren)
val_scores = -ocsvm.decision_function(X_val)

# ROC-Kurve und AUC
fpr, tpr, _ = roc_curve(y_val, val_scores)
roc_auc = roc_auc_score(y_val, val_scores)

# Precision-Recall-Kurve und Average Precision
precision, recall, _ = precision_recall_curve(y_val, val_scores)
pr_auc = average_precision_score(y_val, val_scores)

# Plotten
plt.figure(figsize=(12, 5))

# ROC Curve
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Validation)")
plt.legend()

# PR Curve
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f"AP = {pr_auc:.3f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Validation)")
plt.legend()

plt.tight_layout()
plt.show()


### OCSVM: Anomaly-Scores berechnen

In [ ]:
scores_ocsvm = ocsvm.decision_function(hybrid_features)
print("OCSVM Scores:", scores_ocsvm[:10])

# Modell 3

## DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5)  
dbscan_labels = dbscan.fit_predict(X_test)
dbscan_anomaly = (dbscan_labels == -1).astype(int)
logging.info("\n" + classification_report(y_test, dbscan_anomaly))

### DBSCAN Werte: ARI, HOMO, FMI, Noise Ratio

In [ ]:
ari  = adjusted_rand_score(y_test, dbscan_anomaly)
homo = homogeneity_score(y_test, dbscan_anomaly)
fmi  = fowlkes_mallows_score(y_test, dbscan_anomaly)
noise_ratio = np.sum(dbscan_labels == -1) / len(dbscan_labels)

logging.info(f"Adjusted Rand Index (ARI):     {ari:.4f}")
logging.info(f"Homogeneity:                   {homo:.4f}")
logging.info(f"Fowlkes-Mallows Index (FMI):   {fmi:.4f}")
logging.info(f"Noise Ratio: {noise_ratio:.2f}")

### DBSCAN MCC

In [ ]:
mcc = matthews_corrcoef(y_test, dbscan_anomaly)
logging.info("Matthews Correlation Coefficient: %f", mcc)

### DBSCAN Balanced Accuracy

In [ ]:
balanced_acc = balanced_accuracy_score(y_test, dbscan_anomaly)
logging.info("Balanced Accuracy: %f", balanced_acc)

### DBSCAN AUC-Kurven

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt

# Nutze die binäre Anomalie-Maske als "Score" (0 = normal, 1 = Anomalie)
# → das ergibt eine einzelne Schwelle (wenig Aussagekraft, aber technisch machbar)
scores = dbscan_anomaly  # 0 oder 1

# ROC
fpr, tpr, _ = roc_curve(y_test, scores)
roc_auc = roc_auc_score(y_test, scores)

# Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, scores)
pr_auc = average_precision_score(y_test, scores)

# Plotten
plt.figure(figsize=(12, 5))

# ROC
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (DBSCAN)")
plt.legend()

# PR
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f"AP = {pr_auc:.3f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (DBSCAN)")
plt.legend()

plt.tight_layout()
plt.show()
